# OCR on InfographicVQA dataset via MICROSOFT READ API
#### Results saved as json files. Resizing is performed on images with widht or height greater than 10000 px.

In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
import requests
import json
from pdf2image import convert_from_path
import io

In [15]:
### Authenticates your credentials and creates a client.

subscription_key = ""
endpoint = ""

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

headers = {
    'Ocp-Apim-Subscription-Key': subscription_key, 
    'Content-Type': 'application/octet-stream'
}

In [19]:
for split in ["train", "val", "test"]:
    # Create OCR folder if necessary
    new_ocr_path = f"{split}/{split}_msr_ocr_results"
    if not os.path.exists(new_ocr_path):
        os.makedirs(new_ocr_path)
        
    resized_image_folder = f"{split}/resized_images/"
    if not os.path.exists(resized_image_folder):
        os.makedirs(resized_image_folder)
    
    # Iterate over every infographic
    image_folder = f"{split}/infographicVQA_{split}_v1.0_images"

    for n, image_file in enumerate(sorted(os.listdir(image_folder))):
    #for n, image_file in enumerate(sorted(failed_images[split])):
        image_path = os.path.join(image_folder, image_file)
        is_resized = False
        data = open(image_path, "rb")
        
        image = Image.open(image_path)
        if image.width >= 10000 or image.height >= 10000:
            print(f"{image_file} {split} too large ({image.width}x{image.height}). Resizing")
            
            # The image is too large for the API, resizing is needed
            resized_image = image.resize((min(image.width, 10000), min(image.height, 10000)))
            is_resized = True
            
            # Save the resized image and load its data for the API
            image_path = os.path.join(resized_image_folder, image_file)
            resized_image.save(image_path)
            data = open(image_path, "rb")
            
        # Wait so the request limit is not exceeded
        time.sleep(5)
        # Call to the API
        response = computervision_client.read_in_stream(data, raw=True)
        # Holds the URL used to retrieve the recognized text.
        operation_url = response.headers["Operation-Location"]
        # Take the ID off and use to get results
        operation_id = operation_url.split("/")[-1]

        # The recognized text isn't immediately available, so poll to wait for completion.
        while(True):
            read_result = computervision_client.get_read_result(operation_id)

            if read_result.status.lower () not in ['notstarted', 'running']:
                break
            time.sleep(1)
                
        result_ocr = {}
        result_ocr["status"] = read_result.status
        result_ocr["is_resized"] = is_resized
        result_ocr["recognitionResults"] = []
        
        for text_result in read_result.analyze_result.read_results:
            recognitionResult = {
                "page": text_result.page,
                "clockwiseOrientation": text_result.angle,
                "width": text_result.width,
                "height": text_result.height,
                "unit": text_result.unit.name,
                "lines": []
            }
            result_ocr["recognitionResults"].append(recognitionResult)
            for line in text_result.lines:
                lineResult = {
                    "boundingBox": line.bounding_box,
                    "text": line.text,
                    "words": []
                }
                recognitionResult["lines"].append(lineResult)
                for word in line.words:
                    wordResult = {
                        "boundingBox": word.bounding_box,
                        "text": word.text
                    }
                    lineResult["words"].append(wordResult)
        
        # Write down the ocr results json for the current infographic
        print(f"{n+1}/{len(sorted(os.listdir(image_folder)))}", split, image_file)
        with open(f"{new_ocr_path}/{image_file[:-5]}.json", "w+") as write_file:
            json.dump(result_ocr, write_file)

0/4406 train 10003.jpeg
1/4406 train 10005.jpeg
2/4406 train 10006.jpeg
3/4406 train 10009.jpeg
4/4406 train 10011.jpeg
5/4406 train 10012.jpeg
6/4406 train 10017.jpeg
7/4406 train 10019.jpeg
8/4406 train 10020.jpeg
9/4406 train 10024.jpeg
10/4406 train 10026.jpeg
11/4406 train 10031.jpeg
12/4406 train 10032.jpeg
13/4406 train 10034.jpeg
14/4406 train 10035.jpeg
15/4406 train 10037.jpeg
16/4406 train 10040.jpeg
17/4406 train 10048.jpeg
18/4406 train 10050.jpeg
19/4406 train 10054.jpeg
20/4406 train 10058.jpeg
21/4406 train 10059.jpeg
22/4406 train 10063.jpeg
23/4406 train 10064.jpeg
24/4406 train 10066.jpeg
25/4406 train 10067.jpeg
26/4406 train 10069.jpeg
27/4406 train 10074.jpeg
28/4406 train 10075.jpeg
29/4406 train 10076.jpeg
30/4406 train 10081.jpeg
31/4406 train 10083.jpeg
32/4406 train 10084.jpeg
33/4406 train 10085.jpeg
34/4406 train 10086.jpeg
35/4406 train 10090.jpeg
36/4406 train 10091.jpeg
37/4406 train 10092.jpeg
38/4406 train 10094.jpeg
39/4406 train 10095.jpeg
40/4406 tr

316/4406 train 10696.jpeg
317/4406 train 10697.jpeg
318/4406 train 10699.jpeg
319/4406 train 10700.jpeg
320/4406 train 10701.jpeg
321/4406 train 10704.jpeg
322/4406 train 10705.jpeg
323/4406 train 10706.jpeg
324/4406 train 10712.jpeg
325/4406 train 10715.jpeg
326/4406 train 10719.jpeg
327/4406 train 10722.jpeg
328/4406 train 10725.jpeg
329/4406 train 10726.jpeg
330/4406 train 10729.jpeg
331/4406 train 10732.jpeg
332/4406 train 10738.jpeg
333/4406 train 10740.jpeg
334/4406 train 10741.jpeg
335/4406 train 10744.jpeg
336/4406 train 10747.jpeg
337/4406 train 10748.jpeg
338/4406 train 10749.jpeg
339/4406 train 10750.jpeg
340/4406 train 10751.jpeg
341/4406 train 10753.jpeg
342/4406 train 10756.jpeg
343/4406 train 10757.jpeg
344/4406 train 10759.jpeg
345/4406 train 10763.jpeg
346/4406 train 10764.jpeg
347/4406 train 10766.jpeg
348/4406 train 10767.jpeg
349/4406 train 10772.jpeg
350/4406 train 10775.jpeg
351/4406 train 10777.jpeg
352/4406 train 10778.jpeg
353/4406 train 10779.jpeg
354/4406 tra

630/4406 train 11522.jpeg
631/4406 train 11523.jpeg
632/4406 train 11529.jpeg
633/4406 train 11530.jpeg
634/4406 train 11532.jpeg
635/4406 train 11533.jpeg
636/4406 train 11534.jpeg
637/4406 train 11536.jpeg
638/4406 train 11538.jpeg
639/4406 train 11541.jpeg
640/4406 train 11544.jpeg
641/4406 train 11548.jpeg
642/4406 train 11549.jpeg
643/4406 train 11550.jpeg
644/4406 train 11555.jpeg
645/4406 train 11560.jpeg
646/4406 train 11561.jpeg
647/4406 train 11563.jpeg
648/4406 train 11565.jpeg
649/4406 train 11566.jpeg
650/4406 train 11567.jpeg
651/4406 train 11568.jpeg
652/4406 train 20288.jpeg
653/4406 train 20391.jpeg
654/4406 train 20392.jpeg
655/4406 train 20393.jpeg
656/4406 train 20396.jpeg
657/4406 train 20398.jpeg
658/4406 train 20399.jpeg
659/4406 train 20401.jpeg
660/4406 train 20402.jpeg
20403.jpeg train too large (818x10457). Resizing
661/4406 train 20403.jpeg
662/4406 train 20404.jpeg
663/4406 train 20405.jpeg
664/4406 train 20406.jpeg
665/4406 train 20407.jpeg
666/4406 train 

932/4406 train 30383.jpeg
933/4406 train 30384.jpeg
934/4406 train 30385.jpeg
935/4406 train 30386.jpeg
936/4406 train 30387.jpeg
937/4406 train 30389.jpeg
938/4406 train 30390.jpeg
939/4406 train 30391.jpeg
940/4406 train 30392.jpeg
941/4406 train 30397.jpeg
942/4406 train 30398.jpeg
943/4406 train 30399.jpeg
944/4406 train 30400.jpeg
945/4406 train 30401.jpeg
946/4406 train 30405.jpeg
947/4406 train 30408.jpeg
948/4406 train 30412.jpeg
949/4406 train 30413.jpeg
950/4406 train 30414.jpeg
951/4406 train 30417.jpeg
952/4406 train 30418.jpeg
953/4406 train 30419.jpeg
954/4406 train 30420.jpeg
955/4406 train 30421.jpeg
956/4406 train 30423.jpeg
957/4406 train 30424.jpeg
958/4406 train 30425.jpeg
959/4406 train 30427.jpeg
30428.jpeg train too large (2500x10056). Resizing
960/4406 train 30428.jpeg
961/4406 train 30431.jpeg
962/4406 train 30435.jpeg
963/4406 train 30437.jpeg
964/4406 train 30441.jpeg
965/4406 train 30442.jpeg
966/4406 train 30447.jpeg
967/4406 train 30448.jpeg
968/4406 train

1235/4406 train 30921.jpeg
1236/4406 train 30923.jpeg
1237/4406 train 30924.jpeg
1238/4406 train 30926.jpeg
1239/4406 train 30927.jpeg
1240/4406 train 30929.jpeg
1241/4406 train 30932.jpeg
1242/4406 train 30934.jpeg
1243/4406 train 30937.jpeg
1244/4406 train 30938.jpeg
1245/4406 train 30939.jpeg
1246/4406 train 30940.jpeg
1247/4406 train 30941.jpeg
1248/4406 train 30944.jpeg
1249/4406 train 30947.jpeg
1250/4406 train 30948.jpeg
1251/4406 train 30950.jpeg
1252/4406 train 30951.jpeg
1253/4406 train 30952.jpeg
1254/4406 train 30955.jpeg
1255/4406 train 30956.jpeg
1256/4406 train 30957.jpeg
1257/4406 train 30958.jpeg
1258/4406 train 30962.jpeg
1259/4406 train 30966.jpeg
1260/4406 train 30968.jpeg
1261/4406 train 30969.jpeg
1262/4406 train 30970.jpeg
1263/4406 train 30972.jpeg
1264/4406 train 30974.jpeg
1265/4406 train 30975.jpeg
1266/4406 train 30976.jpeg
1267/4406 train 30978.jpeg
1268/4406 train 30979.jpeg
1269/4406 train 30981.jpeg
1270/4406 train 30982.jpeg
1271/4406 train 30983.jpeg
1

...

3921/4406 train 43410.jpeg
3922/4406 train 43414.jpeg
3923/4406 train 43441.jpeg
3924/4406 train 43451.jpeg
3925/4406 train 43463.jpeg
3926/4406 train 43466.jpeg
3927/4406 train 43484.jpeg
3928/4406 train 43492.jpeg
3929/4406 train 43502.jpeg
3930/4406 train 43504.jpeg
3931/4406 train 43519.jpeg
3932/4406 train 43534.jpeg
3933/4406 train 43536.jpeg
3934/4406 train 43538.jpeg
3935/4406 train 43544.jpeg
3936/4406 train 43547.jpeg
3937/4406 train 43559.jpeg
3938/4406 train 43571.jpeg
3939/4406 train 43572.jpeg
3940/4406 train 43577.jpeg
3941/4406 train 43580.jpeg
3942/4406 train 43584.jpeg
3943/4406 train 43585.jpeg
3944/4406 train 43594.jpeg
3945/4406 train 43599.jpeg
3946/4406 train 43604.jpeg
3947/4406 train 43607.jpeg
3948/4406 train 43608.jpeg
3949/4406 train 43609.jpeg
3950/4406 train 43614.jpeg
3951/4406 train 43622.jpeg
3952/4406 train 43627.jpeg
3953/4406 train 43630.jpeg
3954/4406 train 43633.jpeg
3955/4406 train 43640.jpeg
3956/4406 train 43650.jpeg
3957/4406 train 43652.jpeg
3

4221/4406 train 70056.jpeg
4222/4406 train 70060.jpeg
4223/4406 train 70062.jpeg
4224/4406 train 70067.jpeg
4225/4406 train 70070.jpeg
4226/4406 train 70074.jpeg
4227/4406 train 70076.jpeg
4228/4406 train 70080.jpeg
4229/4406 train 70083.jpeg
4230/4406 train 70089.jpeg
4231/4406 train 70094.jpeg
4232/4406 train 70096.jpeg
4233/4406 train 70097.jpeg
4234/4406 train 70098.jpeg
4235/4406 train 70100.jpeg
4236/4406 train 70103.jpeg
4237/4406 train 70107.jpeg
4238/4406 train 70109.jpeg
4239/4406 train 70111.jpeg
4240/4406 train 70116.jpeg
4241/4406 train 70118.jpeg
4242/4406 train 70136.jpeg
4243/4406 train 70137.jpeg
4244/4406 train 70139.jpeg
4245/4406 train 70140.jpeg
4246/4406 train 70141.jpeg
4247/4406 train 70145.jpeg
4248/4406 train 70149.jpeg
4249/4406 train 70152.jpeg
4250/4406 train 70155.jpeg
4251/4406 train 70159.jpeg
4252/4406 train 70160.jpeg
4253/4406 train 70167.jpeg
4254/4406 train 70169.jpeg
4255/4406 train 70170.jpeg
4256/4406 train 70172.jpeg
4257/4406 train 70178.jpeg
4

140/500 val 20388.jpeg
141/500 val 30076.jpeg
142/500 val 30139.jpeg
143/500 val 30318.jpeg
144/500 val 30341.jpeg
145/500 val 30374.jpeg
146/500 val 30411.jpeg
147/500 val 31122.jpeg
148/500 val 31257.jpeg
149/500 val 31342.jpeg
150/500 val 31539.jpeg
151/500 val 31595.jpeg
152/500 val 31642.jpeg
153/500 val 31963.jpeg
154/500 val 32308.jpeg
155/500 val 32369.jpeg
156/500 val 32433.jpeg
157/500 val 32556.jpeg
158/500 val 32567.jpeg
159/500 val 32807.jpeg
160/500 val 32937.jpeg
161/500 val 33006.jpeg
162/500 val 33088.jpeg
163/500 val 33136.jpeg
164/500 val 33159.jpeg
165/500 val 33364.jpeg
166/500 val 33415.jpeg
167/500 val 33499.jpeg
168/500 val 33505.jpeg
169/500 val 33509.jpeg
170/500 val 33527.jpeg
171/500 val 33555.jpeg
172/500 val 33649.jpeg
173/500 val 33748.jpeg
174/500 val 33877.jpeg
175/500 val 33897.jpeg
176/500 val 33934.jpeg
177/500 val 33961.jpeg
178/500 val 34498.jpeg
179/500 val 34510.jpeg
180/500 val 34642.jpeg
181/500 val 34992.jpeg
182/500 val 35010.jpeg
183/500 val

482/500 val 70138.jpeg
483/500 val 70151.jpeg
484/500 val 70200.jpeg
485/500 val 70206.jpeg
486/500 val 70280.jpeg
487/500 val 70317.jpeg
488/500 val 70344.jpeg
489/500 val 70362.jpeg
490/500 val 70386.jpeg
491/500 val 70405.jpeg
492/500 val 70454.jpeg
493/500 val 70456.jpeg
494/500 val 70504.jpeg
495/500 val 70509.jpeg
496/500 val 70529.jpeg
497/500 val 70539.jpeg
498/500 val 70572.jpeg
499/500 val 70574.jpeg
0/579 test 10002.jpeg
1/579 test 10028.jpeg
2/579 test 10041.jpeg
3/579 test 10045.jpeg
4/579 test 10062.jpeg
5/579 test 10068.jpeg
6/579 test 10070.jpeg
7/579 test 10082.jpeg
8/579 test 10093.jpeg
9/579 test 10111.jpeg
10/579 test 10231.jpeg
11/579 test 10293.jpeg
12/579 test 10297.jpeg
13/579 test 10310.jpeg
14/579 test 10319.jpeg
15/579 test 10323.jpeg
16/579 test 10325.jpeg
17/579 test 10360.jpeg
18/579 test 10372.jpeg
19/579 test 10402.jpeg
20/579 test 10403.jpeg
21/579 test 10410.jpeg
22/579 test 10412.jpeg
23/579 test 10437.jpeg
24/579 test 10454.jpeg
25/579 test 10473.jpe

327/579 test 37354.jpeg
328/579 test 37355.jpeg
329/579 test 37359.jpeg
330/579 test 37365.jpeg
331/579 test 37378.jpeg
332/579 test 37382.jpeg
333/579 test 37383.jpeg
334/579 test 37387.jpeg
335/579 test 37390.jpeg
336/579 test 37393.jpeg
337/579 test 37398.jpeg
338/579 test 37459.jpeg
339/579 test 37544.jpeg
340/579 test 37545.jpeg
341/579 test 39282.jpeg
342/579 test 39284.jpeg
39327.jpeg test too large (2500x14669). Resizing
343/579 test 39327.jpeg
344/579 test 39340.jpeg
345/579 test 39365.jpeg
346/579 test 39381.jpeg
347/579 test 39464.jpeg
348/579 test 39510.jpeg
349/579 test 39511.jpeg
350/579 test 39545.jpeg
351/579 test 39551.jpeg
352/579 test 39554.jpeg
353/579 test 39556.jpeg
354/579 test 39592.jpeg
355/579 test 39628.jpeg
356/579 test 39632.jpeg
357/579 test 39633.jpeg
358/579 test 39635.jpeg
359/579 test 39638.jpeg
360/579 test 39679.jpeg
361/579 test 39681.jpeg
362/579 test 39685.jpeg
363/579 test 39687.jpeg
364/579 test 39727.jpeg
365/579 test 39753.jpeg
366/579 test 39

In [5]:
# Infographics with width or height >= 10000
large_images = {
    'train': [
        '10133.jpeg',
        '10611.jpeg',
        '10868.jpeg',
        '20403.jpeg',
        '20411.jpeg',
        '20464.jpeg',
        '20467.jpeg',
        '30032.jpeg',
        '30124.jpeg',
        '30361.jpeg',
        '30428.jpeg',
        '30616.jpeg',
        '31166.jpeg',
        '31416.jpeg',
        '31748.jpeg',
        '31912.jpeg',
        '32199.jpeg',
        '32494.jpeg',
        '33681.jpeg',
        '33838.jpeg',
        '33840.jpeg',
        '33909.jpeg',
        '35333.jpeg',
        '35443.jpeg',
        '35471.jpeg',
        '37949.jpeg',
        '38019.jpeg',
        '38040.jpeg',
        '38238.jpeg',
        '38361.jpeg',
        '38377.jpeg',
        '40253.jpeg',
        '40315.jpeg',
        '40617.jpeg',
        '41454.jpeg',
        '41567.jpeg',
        '41997.jpeg',
        '42051.jpeg',
        '44048.jpeg',
        '44804.jpeg',
        '70201.jpeg'
     ],
     'test': [
         '11159.jpeg',
         '39327.jpeg',
         '40353.jpeg'
     ],
     'val': [
        '20369.jpeg',
        '37212.jpeg',
        '37324.jpeg',
        '37395.jpeg',
        '39522.jpeg',
        '39695.jpeg',
        '44728.jpeg',
        '44787.jpeg'
     ]
}